In [13]:
import os
import requests
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
#from dotenv import load_dotenv

#from db import init_db

In [19]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"

In [20]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [21]:
df = df.iloc[:300]
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


## Q1. Getting the enbeddings model

In [22]:
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

### Create the embeddings for the first LLM answer

In [23]:
answer_llm = df.iloc[0].answer_llm

### What's the first value of the resulting vector?

In [25]:
first_answer_v = embedding_model.encode(answer_llm)
first_answer_v[0]

-0.42244676

## Q2. Computing the dot product

In [26]:
answers_llm = df.to_dict(orient='records')

In [27]:
answers_llm[0]

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [28]:
def compute_dot_product(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']

    v_orig = embedding_model.encode(answer_orig)
    v_llm = embedding_model.encode(answer_llm)

    return v_orig.dot(v_llm)


In [29]:
evaluations = [] 

for record in tqdm(answers_llm):
    dot_product = compute_dot_product(record)
    evaluations.append(dot_product)

  0%|          | 0/300 [00:00<?, ?it/s]

In [30]:
evaluations[:10]

[17.515995,
 13.418406,
 25.31325,
 12.147421,
 18.747725,
 33.970398,
 30.251698,
 29.52158,
 35.2722,
 27.751759]

### What's the 75% of the score? 

In [31]:
df['dot_product'] = evaluations 
df['dot_product'].describe()


count    300.000000
mean      27.495996
std        6.384743
min        4.547925
25%       24.307844
50%       28.336862
75%       31.674305
max       39.476013
Name: dot_product, dtype: float64

In [33]:
print(f"75% of the score is {df['dot_product'].describe()['75%']}")

75% of the score is 31.67430543899536


## Q3. Computing the cosine

In [35]:
import numpy as np

In [36]:
# Normalize a vector 
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm 
    return v_norm 

In [37]:
def compute_dot_product_normalized(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']

    v_orig = embedding_model.encode(answer_orig)
    v_llm = embedding_model.encode(answer_llm)

    v_orig_normalized = normalize_vector(v_orig)
    v_llm_normalized = normalize_vector(v_llm)
    
    return v_orig_normalized.dot(v_llm_normalized)

In [38]:
normalized_evaluations = [] 

for record in tqdm(answers_llm):
    n_dot_product = compute_dot_product_normalized(record)
    normalized_evaluations.append(n_dot_product)

  0%|          | 0/300 [00:00<?, ?it/s]

In [39]:
normalized_evaluations[:10]

[0.5067542,
 0.38854897,
 0.7185991,
 0.33726645,
 0.5217922,
 0.83053195,
 0.7462834,
 0.6944062,
 0.8468864,
 0.65590733]

### What's the 75% cosine in the scores? 

In [40]:
df['n_dot_product'] = normalized_evaluations 
df['n_dot_product'].describe()

count    300.000000
mean       0.728392
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: n_dot_product, dtype: float64

In [41]:
print(f"75% of cosine scores is {df['n_dot_product'].describe()['75%']}")

75% of cosine scores is 0.8362347185611725


## Q4. Rouge 

In [42]:
from rouge import Rouge
rouge_scorer = Rouge()

In [46]:
answers_llm[10]

{'answer_llm': "Yes, all sessions are recorded, so if you miss one, you won't miss anything. You can catch up on the content later. Additionally, you can submit your questions in advance for office hours, and those sessions are also recorded.",
 'answer_orig': 'Everything is recorded, so you won’t miss anything. You will be able to ask your questions for office hours in advance and we will cover them during the live stream. Also, you can always ask questions in Slack.',
 'document': '5170565b',
 'question': 'Are sessions recorded if I miss one?',
 'course': 'machine-learning-zoomcamp'}

In [ ]:
#Let's compute the ROUGE score between the answers at the index 10 of our dataframe (doc_id=5170565b)
record = answers_llm[10]
scores = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])

In [45]:
print(scores)

[{'rouge-1': {'r': 0.45454545454545453, 'p': 0.45454545454545453, 'f': 0.45454544954545456}, 'rouge-2': {'r': 0.21621621621621623, 'p': 0.21621621621621623, 'f': 0.21621621121621637}, 'rouge-l': {'r': 0.3939393939393939, 'p': 0.3939393939393939, 'f': 0.393939388939394}}]


### What's the F score for rouge-1?

In [47]:
print(f"F-score for rouge-1 = {scores[0]['rouge-1']['f']} ")

F-score for rouge-1 = 0.45454544954545456 


## Q5. Average Rouge Score 

In [50]:
# given a rouge score compute the average of all 3 scores 
def average_rouge_score(rouge_score_record):
    rouge_1 = rouge_score_record['rouge-1']['f']
    rouge_2 = rouge_score_record['rouge-2']['f']
    rouge_l = rouge_score_record['rouge-l']['f']
    rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
    return rouge_avg

In [51]:
# Let's compute the average between rouge-1, rouge-2 and rouge-l for the same record from Q4
avg_score = average_rouge_score(scores[0])
print(f"average between rouge-1, rouge-2 and rouge-l for the same record from Q4: {avg_score}")

average between rouge-1, rouge-2 and rouge-l for the same record from Q4: 0.35490034990035496


## Q6. Average rouge score for all the data points


In [52]:
avg_scores = []
for record in tqdm(answers_llm):
    score = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])
    avg = average_rouge_score(score[0])
    avg_scores.append(avg)

  0%|          | 0/300 [00:00<?, ?it/s]

In [53]:
df_avg = pd.DataFrame({'avg_rouge_scores': avg_scores})

In [56]:
df_avg.head()

,avg_rouge_scores
0,0.072882
1,0.091435
2,0.327658
3,0.150821
4,0.098731


In [57]:
df_avg.describe()

,avg_rouge_scores
count,300.000000
mean,0.313205
std,0.158133
min,0.000000
25%,0.197358
50%,0.298640
75%,0.404169
max,0.813043


In [59]:
# What's the average rouge_l across all the nodes? 
print(f"Average rouge score across all the records is {df_avg['avg_rouge_scores'].mean()}")

Average rouge score across all the records is 0.313205367339838
